In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh


In [2]:
import sys
sys.path.append("..")

# MAKE SURE TO SET-UP PATH -> use local to run with demo data; use azure to run with complete dataset (access required)
# Select where to run notebook: "azure" or "local"
my_run = "azure"

if my_run == "azure":
    import config_azure as cf
    running_demo = False
elif my_run == "local":
    import config as cf
    running_demo = True


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE

import pandas as pd

In [3]:
import torch
torch.cuda.empty_cache()

## Notebook Overview
Goal: Fine-tune models for document classification.

Method: the documents are shortened by taking the first 200 tokens. Then the shortened doc is formatted using the zero-shot prompt, without template. Then the ideal response is formatted according to JSON format. Formatted doc and response are combined into conversation using the apply_chat_template function. 

*Previous notebook: FinetuningDataFormatting*

*Next notebook: GetPredictions*

NOTE: fine-tuning requires a bigger compute than promping them. We used the Standard_NC24ads_A100_v4m instance with 24 cores and 220GB RAM, which is powered by NVIDIA A100 PCIe GPU.

In [4]:
# necesarry to log in to huggingface, to save models there
from huggingface_hub import notebook_login
notebook_login()

### Finetuning GEITje

In [4]:
# Load GEITje formatted data

from datasets import load_dataset
chat_dataset = load_dataset('FemkeBakker/AmsterdamBalancedFirst200Tokens')


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# basemodel_name = 'Rijgersberg/GEITje-7B-chat-v2'
basemodel_name = 'mistralai/Mistral-7B-Instruct-v0.2'
# basemodel_name = "stabilityai/stablelm-2-1_6b"
# basemodel_name = 'meta-llama/Llama-2-7b-chat-hf'
model = AutoModelForCausalLM.from_pretrained(basemodel_name, torch_dtype=torch.bfloat16,
                                                low_cpu_mem_usage=True, attn_implementation="sdpa",
                                                device_map='cpu')


tokenizer = AutoTokenizer.from_pretrained(basemodel_name)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = 'right'

model.config.pad_token_id = tokenizer.unk_token_id

/anaconda/envs/2AmsterdamLLM/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
import pandas as pd

# funcation to load previous saved dataframe and combine with current model, then save again
def combine_and_save_df(model_df, save_to_path):
    
    # combine with earlier runs if exists
    if os.path.exists(save_to_path):
        original = pd.read_pickle(save_to_path)
        model_df = pd.concat([original, model_df])

    model_df.to_pickle(save_to_path)

In [14]:
import torch
from datasets import DatasetDict, load_dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
import time

import sys
sys.path.append('../src/') 
import prediction_helperfunctions as ph

def train(model, model_name, tokenizer, chat_dataset, chat_dataset_name, new_model_name, output_directory, train_set, test_set, n_epochs, run_id='No_id', save_to_hub=True, resume=False):
    start_time = time.time()

    # format conversations
    def format(examples):
        return [tokenizer.apply_chat_template(conversation, tokenize=False)
                for conversation in examples['message']]

    per_device_train_batch_size = 2
    gradient_accumulation_steps = 8
    steps_per_epoch = len(chat_dataset[train_set])\
                // (torch.cuda.device_count() * per_device_train_batch_size * gradient_accumulation_steps)
    eval_steps = steps_per_epoch // 5

    training_args = TrainingArguments(
        optim='adamw_bnb_8bit',
        num_train_epochs=n_epochs,
        learning_rate=1e-5,
        lr_scheduler_type='cosine',
        warmup_ratio=0.1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        gradient_checkpointing=True,
        evaluation_strategy='steps',
        eval_steps=eval_steps,
        save_strategy='epoch',
        bf16=False, #bf16=True require CUDA 11 -> original code bf16=True
        output_dir=output_directory,
        report_to=["tensorboard", 'wandb'],
        logging_steps=1,
        logging_first_step=True,
        hub_model_id=new_model_name,
        push_to_hub=True,
        hub_private_repo=True,
        hub_strategy='all_checkpoints',
    )

    trainer = SFTTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        max_seq_length=8192,
        train_dataset=chat_dataset[train_set],
        eval_dataset=chat_dataset[test_set],
        formatting_func=format,
        neftune_noise_alpha=5,
    )

  
    dict_info = {
        'model':new_model_name,
        'base_model':model_name,
        'chat_dataset':chat_dataset_name,
        'train_set':train_set,
        'test_set': test_set,
        'training_args': training_args,
        'resume_from_checkpoint':resume,
        'date':ph.get_datetime(),
        'runtime': False,
        'Error': False,
        'run_id':run_id,
        'save_to_hub':save_to_hub,
        'output_dir': output_directory,
        'num_train_epochs':n_epochs
        }

    data = pd.DataFrame(columns=dict_info.keys())

    # if no error during training, save run in overview_models and push to hub
    try:
        trainer.train(resume_from_checkpoint=resume)
        if save_to_hub == True:
            trainer.push_to_hub()
            
        dict_info['runtime'] = time.time()-start_time

        data.loc[len(data)] = dict_info
        combine_and_save_df(data, f'{cf.output_path}/finetuning_output/overview_models.pkl')
        print("Finished without error!")

    # if keyboardinterrupted or an error is thrown, save run in overview_models
    except KeyboardInterrupt:
        dict_info['Error'] = 'KeyboardInterrupt'        
        dict_info['runtime'] = time.time()-start_time

        data.loc[len(data)] = dict_info
        combine_and_save_df(data, f'{cf.output_path}/finetuning_output/overview_models.pkl')

    except Exception  as e:
        print(e)
        dict_info['Error'] = e
        dict_info['runtime'] = time.time()-start_time

        data.loc[len(data)] = dict_info
        combine_and_save_df(data, f'{cf.output_path}/finetuning_output/overview_models.pkl')


        model_df = pd.DataFrame(dict_info)
        combine_and_save_df(model_df, f'{cf.output_path}/finetuning_output/overview_models.pkl')


**Note**

To use resume_from_checkpoint, the epoch must be complete; otherwise, it will throw an error. If an error occurs even after an epoch is complete, remove the last checkpoint folder to resolve this. This means you can only resume training from a completed checkpoint. Since each epoch took about 50 minutes, this was not an issue.

MAKE SURE: run_id is unique, for each seperate run. Check overview_models.pkl to find which run ids have already been used.

In [15]:
# path to folder where the checkpoint of the model need to be saved
output_directory = f'{cf.output_path}/finetuning_output/MistralTry2epochs'

# the name of the chat dataset
chat_dataset_name = 'FemkeBakker/AmsterdamBalancedFirst200Tokens'

training_set = 'train' 
validation_set = 'val'

new_model_name = 'FemkeBakker/MistralTry2epochs'
n_epochs = 2

In [16]:
train(model, basemodel_name, tokenizer, chat_dataset, chat_dataset_name,new_model_name,
          output_directory, training_set, validation_set,  n_epochs, run_id=31, save_to_hub=True, resume=False)

/anaconda/envs/2AmsterdamLLM/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/anaconda/envs/2AmsterdamLLM/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:257: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: femkebakker (thesisamsterdam). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/anaconda/envs/2AmsterdamLLM/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
123,0.986300,0.879042
246,0.791800,0.832382
369,0.513300,0.791489
492,0.570200,0.759139
615,0.789700,0.697555
738,0.587200,0.676763
861,0.424200,0.664945
984,0.522200,0.660889
1107,0.260900,0.659912
1230,0.483400,0.660117


/anaconda/envs/2AmsterdamLLM/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Finished without error!
